In [3]:
import sys
sys.path.append('../')
import util

/iris/u/nathu/un_camels/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import pickle
import os
import numpy as np
from tqdm import tqdm


In [2]:
with open('/iris/u/nathu/temporal-LMs/learned_updating/data/NYT_aggregated_info.pkl', 'rb') as f:
    NYT_aggregated_info = pickle.load(f)

In [3]:
#drop rows with no text

#make sure we were able to correctly parse the date
NYT_aggregated_info['year'] = NYT_aggregated_info['date'].apply(lambda x: None if x.split('-')[0] == 'None' else int(x.split('-')[0]))
NYT_aggregated_info = NYT_aggregated_info.dropna(subset=['year', 'text', 'title'])
NYT_aggregated_info = NYT_aggregated_info[NYT_aggregated_info['year'] <= 2008]
NYT_aggregated_info['date'] = pd.to_datetime(NYT_aggregated_info['date'])
NYT_aggregated_info.sort_values(by=['date'], inplace=True)

In [4]:
def gen_merged_text(text):
    if text[0][:5] == 'LEAD:':
        return '\n'.join(text[1:])
    else:
        return '\n'.join(text)

In [5]:
NYT_aggregated_info['merged_text'] = NYT_aggregated_info['text'].apply(gen_merged_text)

In [7]:
NYT_aggregated_info['avg_para_len'] = NYT_aggregated_info.apply(lambda x: np.mean([len(para) for para in x['text']]), axis = 1)
NYT_aggregated_info['len(para)'] = NYT_aggregated_info['text'].apply(lambda x: len(x))

print(len(NYT_aggregated_info))
#filter out paid notices/obituatries
NYT_aggregated_info = NYT_aggregated_info[~NYT_aggregated_info['title'].apply(lambda x: x[:12] == 'Paid Notice:')]
print('filtered paid_notices', len(NYT_aggregated_info))
#earnings reports
NYT_aggregated_info = NYT_aggregated_info[~NYT_aggregated_info.apply(lambda x: '*** COMPANY REPORTS **' in x['merged_text'], axis = 1)]
print('filtered earnings reports', len(NYT_aggregated_info))
NYT_aggregated_info = NYT_aggregated_info[~NYT_aggregated_info.apply(lambda x: 'To the Editor:' in x['merged_text'], axis = 1)]
print('filtered "to the editor:"', len(NYT_aggregated_info))
NYT_aggregated_info = NYT_aggregated_info[NYT_aggregated_info.apply(lambda x: len(x['merged_text']) > 500, axis = 1)]
print('filtered for 500+ char', len(NYT_aggregated_info))
text_para_lens = NYT_aggregated_info['text'].apply(lambda x: len(x))

NYT_aggregated_info = NYT_aggregated_info[NYT_aggregated_info['avg_para_len'] > 150]
print('filtered for 150+ char avg para len', len(NYT_aggregated_info))

NYT_aggregated_info = NYT_aggregated_info[NYT_aggregated_info['len(para)'] <= 50]
print('filtered for <=50 para articles', len(NYT_aggregated_info))

1829595
filtered paid_notices 1698297
filtered earnings reports 1595048
filtered "to the editor:" 1481243
filtered for 500+ char 1345750
filtered for 150+ char avg para len 1286682
filtered for <=50 para articles 1272095


In [22]:
NYT_aggregated_info['text'] = NYT_aggregated_info['merged_text']
NYT_aggregated_info = NYT_aggregated_info.drop(columns=['merged_text', 'avg_para_len', 'len(para)', 'year'])

In [23]:
qa_pretrain_split = NYT_aggregated_info[NYT_aggregated_info['date'] <= pd.to_datetime('1988-09-30')]
qa_pretrain_val_split = NYT_aggregated_info[(NYT_aggregated_info['date'] >= pd.to_datetime('1988-10-01'))
                                            &(NYT_aggregated_info['date'] <= pd.to_datetime('1988-12-31'))]
meta_train_split = NYT_aggregated_info[(NYT_aggregated_info['date'] > pd.to_datetime('1989-01-01')) 
                                       & (NYT_aggregated_info['date'] <= pd.to_datetime('1992-12-31'))]
eval_split = NYT_aggregated_info[(NYT_aggregated_info['date'] > pd.to_datetime('1993-01-01')) 
                                 & (NYT_aggregated_info['date'] <= pd.to_datetime('1994-12-31'))]
unused_split = NYT_aggregated_info[(NYT_aggregated_info['date'] >= pd.to_datetime('1995-01-01'))]
                                   
print('qa_pretrain_split', len(qa_pretrain_split))
print('qa_pretrain_val_split', len(qa_pretrain_val_split))
print('meta_train_split', len(meta_train_split))
print('eval_split', len(eval_split))
print('unused_split', len(unused_split))




qa_pretrain_split 124836
qa_pretrain_val_split 18256
meta_train_split 260324
eval_split 110276
unused_split 757990


In [32]:
qa_pretrain_split.to_csv('../data/nyt/qa_pretrain_split.csv')
qa_pretrain_val_split.to_csv('../data/nyt/qa_pretrain_val_split.csv')
meta_train_split.to_csv('../data/nyt/meta_train_split.csv')
eval_split.to_csv('../data/nyt/eval_split.csv')
unused_split.to_csv('../data/nyt/unused_split.csv')

In [33]:
meta_train_split

,title,date,text,doc_id
834040,"Reviews/Television; How Washington Works, as S...",1989-01-02,Like Hedrick Smith's knowing book from which i...,211071
834044,The Video News Release: Let the Viewer Beware,1989-01-02,WHEN the ''CBS Evening News With Dan Rather'' ...,211197
834043,Israel Expels 13 Palestinians Accused of Aidin...,1989-01-02,Thirteen Palestinians accused of leading the u...,211142
834042,Two Old Rivals Meet in Pasadena,1989-01-02,Bo Schembechler has mellowed so much that he i...,211114
834041,Brown Wants to Lead Party He Once Spurned,1989-01-02,Once hailed by some as the most inspired polit...,211120
...,...,...,...,...
1056879,THE MEDIA BUSINESS: Advertising -- Addenda; Th...,1992-12-31,Northwest Airlines said that it had dismissed ...,579960
1056880,Music Notes,1992-12-31,Ecological Correctness\nEnvironmentalism has d...,580064
1056881,"Calendar: Workshops, Classes, Tours and a Sale",1992-12-31,"Family Workshops\nWave Hill, Independence Aven...",580056
1056882,U.N. Drafts Asylum Plan for Haitian Boat People,1992-12-31,The United Nations High Commissioner for Refug...,579983
